# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
#
import re

In [2]:
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download(['punkt', 'stopwords', 'wordnet'])

stop_words = stopwords.words('english')

[nltk_data] Downloading package punkt to /Users/jom/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/jom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jom/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
# Pipeline
from sklearn.pipeline import Pipeline
from joblib import dump, load


In [4]:
# load data from database
engine = create_engine('sqlite:///database.db')
df = pd.read_sql_table('categories', engine)
X = df['message']
Y = df.iloc[:, 4:]

In [5]:
lemmatizer = WordNetLemmatizer()

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    """ Extract words from message, lower text, remove punctuation and stopwords
    """
    # remove non alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())
    # tokenize text
    tokens = word_tokenize(text)
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3)

In [10]:
pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', DecisionTreeRegressor()),
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', DecisionTreeRegressor())])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = pipeline.predict(X_test)

In [11]:
y_test.iloc[:, 0].astype(int).values

array([0, 1, 1, ..., 1, 1, 1])

In [12]:
y_pred[:, 0]

array([0., 1., 1., ..., 1., 1., 1.])

In [13]:
print(classification_report(y_test.iloc[:, 3].values, y_pred[:, 3].astype(int), target_names=['False', 'True']))

              precision    recall  f1-score   support

       False       0.71      0.79      0.75      4572
        True       0.65      0.54      0.59      3293

    accuracy                           0.69      7865
   macro avg       0.68      0.67      0.67      7865
weighted avg       0.68      0.69      0.68      7865



In [14]:
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

In [15]:
f1 = [f1_score(y_test.iloc[:, i].values, y_pred[:, i].astype(int)) for i in range(y_test.shape[1])]
recall = [recall_score(y_test.iloc[:, i].values, y_pred[:, i].astype(int)) for i in range(y_test.shape[1])]


/opt/miniconda3/envs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/opt/miniconda3/envs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
no_tuned_score = pd.DataFrame({
    'column':list(y_test.columns),
    'f1_score':f1,
    'recall_score':recall,
})

In [17]:
no_tuned_score

,column,f1_score,recall_score
0,related,0.827873,0.801743
1,request,0.548686,0.516739
2,offer,0.040816,0.034483
3,aid_related,0.593465,0.543274
4,medical_help,0.204739,0.162896
5,medical_products,0.217586,0.178049
6,search_and_rescue,0.168067,0.127660
7,security,0.017316,0.013889
8,military,0.201207,0.188679
9,child_alone,0.000000,0.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
# {'ccp_alpha': 0.0,
#  'criterion': 'mse',
#  'max_depth': None,
#  'max_features': None,
#  'max_leaf_nodes': None,
#  'min_impurity_decrease': 0.0,
#  'min_impurity_split': None,
#  'min_samples_leaf': 1,
#  'min_samples_split': 2,
#  'min_weight_fraction_leaf': 0.0,
#  'random_state': None,
#  'splitter': 'best'}


parameters={"clf__splitter":["best","random"],
            "clf__max_depth" : [None,1,9],
#            "clf__min_samples_leaf":[1,2],
           "clf__min_weight_fraction_leaf":[0.,0.1],
           "clf__max_features":["log2","sqrt",None],
           "clf__max_leaf_nodes":[None,20,50] }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=1)

In [15]:
dir(cv)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_is_fitted',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_pairwise',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_validate_data',
 'classes_',
 'cv',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'inverse_transform',
 'n_features_in_',
 'n_jobs',
 'param_grid',
 'pre_dispatch',
 'predict',
 'predict_log_proba',
 'predict_proba',
 'refit',
 'return_train_sco

In [27]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', DecisionTreeRegressor())]),
             param_grid={'clf__max_depth': [None, 1, 9],
                         'clf__max_features': ['log2', 'sqrt', None],
                         'clf__max_leaf_nodes': [None, 20, 50],
                         'clf__min_weight_fraction_leaf': [0.0, 0.1],
                         'clf__splitter': ['best', 'random']},
             verbose=1)

In [28]:
cv.best_params_

{'clf__max_depth': None,
 'clf__max_features': None,
 'clf__max_leaf_nodes': 50,
 'clf__min_weight_fraction_leaf': 0.0,
 'clf__splitter': 'best'}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', DecisionTreeRegressor(max_depth=None, max_features=None,
#              max_leaf_nodes=20,
             min_samples_leaf=1,
             min_weight_fraction_leaf=0.,
             splitter='best'))
        ])

In [19]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', DecisionTreeRegressor())])

In [20]:
y_pred = pipeline.predict(X_test)

In [21]:
f1 = [f1_score(y_test.iloc[:, i].values, y_pred[:, i].round().astype(int)) for i in range(y_test.shape[1])]
recall = [recall_score(y_test.iloc[:, i].values, y_pred[:, i].astype(int)) for i in range(y_test.shape[1])]

/opt/miniconda3/envs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/opt/miniconda3/envs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
tuned_score = pd.DataFrame({
    'column':list(y_test.columns),
    'f1_score':f1,
    'recall_score':recall,
})

In [23]:
tuned_score

,column,f1_score,recall_score
0,related,0.836220,0.817524
1,request,0.553646,0.527656
2,offer,0.000000,0.000000
3,aid_related,0.600263,0.555421
4,medical_help,0.215014,0.174962
5,medical_products,0.234957,0.200000
6,search_and_rescue,0.159780,0.123404
7,security,0.008197,0.006944
8,military,0.227451,0.218868
9,child_alone,0.000000,0.000000


In [24]:
y_pred[:, 1]

array([0., 0., 0., ..., 0., 0., 0.])

In [25]:
no_tuned_score.merge(tuned_score, on='column', how='left')

,column,f1_score_x,recall_score_x,f1_score_y,recall_score_y
0,related,0.827873,0.801743,0.836220,0.817524
1,request,0.548686,0.516739,0.553646,0.527656
2,offer,0.040816,0.034483,0.000000,0.000000
3,aid_related,0.593465,0.543274,0.600263,0.555421
4,medical_help,0.204739,0.162896,0.215014,0.174962
5,medical_products,0.217586,0.178049,0.234957,0.200000
6,search_and_rescue,0.168067,0.127660,0.159780,0.123404
7,security,0.017316,0.013889,0.008197,0.006944
8,military,0.201207,0.188679,0.227451,0.218868
9,child_alone,0.000000,0.000000,0.000000,0.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.